In [ ]:
import pandas as pd, re, os
from pathlib import Path

# 1) train.csv → level/grade 파싱
df = pd.read_csv('./train.csv')
long = df.melt('study_id', var_name='key', value_name='grade')

# level 추출
long[['junk', 'level']] = long['key'].str.extract(r'(.*)_(l\d_l\d|l5_s1)$', expand=True)
long['level'] = long['level'].str.upper().str.replace('L', 'L').str.replace('_', '_')

# 이진 라벨
long['is_abnormal'] = long['grade'].isin(['Moderate', 'Severe']).astype(int)

# 레벨별 OR
disc = long.groupby(['study_id','level'], as_index=False)['is_abnormal'].max()
disc['filename'] = disc['study_id'].astype(str)+'_'+disc['level']+'.jpg'

# 2) RSNA crop 목록
crop_files = [p.name for p in Path('crops_rsna').glob('*.jpg')]
crop_df = pd.DataFrame({'filename': crop_files})

# 3) 병합 → NaN = 0
labels = (crop_df
          .merge(disc[['filename','is_abnormal']], on='filename', how='left')
          .fillna({'is_abnormal':0}).astype({'is_abnormal':int}))

labels.to_csv('train_labels.csv', index=False)
print(f'라벨링 완료: {len(labels)}개')

라벨링 완료: 0개
